<a href="https://colab.research.google.com/github/PhiDCH/CRAFT-Reimplementation/blob/master/CRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/craft-pytorch
!git clone https://github.com/PhiDCH/CRAFT-pytorch /content/craft-pytorch

# Get backbones VGG16 and pretrained model

In [6]:
!rm -rf /content/craft-pytorch/pretrain/
!mkdir /content/craft-pytorch/pretrain/

In [ ]:
!gdown --id 1ZtvGpFQrbmEisB_GhmZb8UQOtvqY_-tW -O /content/craft-pytorch/pretrain/vgg16_bn.pth
!gdown --id 1i2R7UIUqmkUtF0jv_3MXTqmQ_9wuAnLf -O /content/craft-pytorch/pretrain/clovaAI_ic15.pth
!gdown --id 1k17GuBG_omT91tJoIMSlLrorYbLXkq4z -O /content/craft-pytorch/pretrain/backtime92_ic15.pth

# Test for some pictures


In [40]:
cd /content/craft-pytorch

/content/craft-pytorch


In [42]:
import torch
from craft import CRAFT
from collections import OrderedDict

def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

net = CRAFT().cuda()

In [44]:
import time 

t = time.time()
net.load_state_dict(copyStateDict(torch.load("./pretrain/backtime92_ic15.pth")))
net.cuda()
net(torch.randn(1,3,768,768).cuda())
print(time.time()-t)

t = time.time()
net.load_state_dict(copyStateDict(torch.load("./pretrain/clovaAI_ic15.pth")))
net.cuda()
net(torch.randn(1,3,768,768).cuda())
print(time.time()-t)

0.2729640007019043
0.19815707206726074


In [ ]:
!python test.py --help

In [ ]:
!python test.py --trained_model="./pretrain/clovaAI_ic15.pth" --test_folder="./test_img/" --show_time --poly

# prepare data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir /content/icdar2015
!mkdir /content/icdar2015/train_img
!mkdir /content/icdar2015/train_gt
!mkdir /content/icdar2015/test_img
!mkdir /content/icdar2015/test_gt
!unzip /content/drive/MyDrive/MyComputer/icdar2015/ch4_training_images.zip -d /content/icdar2015/train_img
!unzip /content/drive/MyDrive/MyComputer/icdar2015/ch4_training_localization_transcription_gt.zip -d /content/icdar2015/train_gt
!unzip /content/drive/MyDrive/MyComputer/icdar2015/ch4_test_images.zip -d /content/icdar2015/test_img
!unzip /content/drive/MyDrive/MyComputer/icdar2015/Challenge4_Test_Task1_GT.zip -d /content/icdar2015/test_gt

# evaluation ICDAR_2015

In [5]:
!rm -rf /content/craft-pytorch
!git clone https://github.com/PhiDCH/CRAFT-Reimplementation /content/craft-pytorch

Cloning into '/content/craft-pytorch'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 227 (delta 3), reused 0 (delta 0), pack-reused 217
Receiving objects: 100% (227/227), 136.43 MiB | 32.37 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [1]:
cd /content/craft-pytorch/

/content/craft-pytorch


In [48]:
!mkdir /content/icdar2015/result
!mkdir /content/icdar2015/result/gt
!mkdir /content/icdar2015/result/mask

In [ ]:
import torch
from craft import CRAFT
from collections import OrderedDict

def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

net = CRAFT().cuda()
net.load_state_dict(copyStateDict(torch.load("./pretrain/clovaAI_ic15.pth")))
net.cuda()

In [3]:
from evaluation import eval2015 

In [8]:
pip install Polygon3

     |████████████████████████████████| 71kB 5.1MB/s 
  Created wheel for Polygon3: filename=Polygon3-3.0.8-cp37-cp37m-linux_x86_64.whl size=102403 sha256=e41114174a7fdebbfceefbd9f74a2afa5f598f2e002953a1b34851d15e192498
  Stored in directory: /root/.cache/pip/wheels/95/32/f1/5525b233996d9d99cbce2f0a8da60d137ddddc555d3e8b0e2a
Successfully built Polygon3


In [4]:
eval2015(net, '/content/icdar2015/test_img', '/content/icdar2015/result')

Test image 1/500: /content/icdar2015/test_img/img_31.jpg
Test image 2/500: /content/icdar2015/test_img/img_140.jpg


/content/craft-pytorch/craft_utils.py:242: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  polys = np.array(polys)


Test image 3/500: /content/icdar2015/test_img/img_493.jpg
Test image 4/500: /content/icdar2015/test_img/img_186.jpg
Test image 5/500: /content/icdar2015/test_img/img_341.jpg
Test image 6/500: /content/icdar2015/test_img/img_425.jpg
Test image 7/500: /content/icdar2015/test_img/img_1.jpg
Test image 8/500: /content/icdar2015/test_img/img_129.jpg
Test image 9/500: /content/icdar2015/test_img/img_84.jpg
Test image 10/500: /content/icdar2015/test_img/img_475.jpg
Test image 11/500: /content/icdar2015/test_img/img_285.jpg
Test image 12/500: /content/icdar2015/test_img/img_402.jpg
Test image 13/500: /content/icdar2015/test_img/img_160.jpg
Test image 14/500: /content/icdar2015/test_img/img_159.jpg
Test image 15/500: /content/icdar2015/test_img/img_156.jpg
Test image 16/500: /content/icdar2015/test_img/img_421.jpg
Test image 17/500: /content/icdar2015/test_img/img_410.jpg
Test image 18/500: /content/icdar2015/test_img/img_403.jpg
Test image 19/500: /content/icdar2015/test_img/img_38.jpg
Test ima